# Roguelike Universe - Link Generation

We can generate the inspiration network based on our data.

In [521]:
import os
import io
import json
import pandas as pd

def read_json(path):
    data = ''
    with io.open(path, 'r', encoding='utf-8') as f:
        data = json.loads(f.read())
#         print(__message('Loaded {}'.format(path)))
    return data
    
def save_json(path, data):
    with io.open(path, 'w', encoding='utf-8') as f:
        try:
            output = json.dumps(data, indent=2, ensure_ascii=False)
            f.write(output)
        except UnicodeEncodeError:
            f.write(output.encode('utf-8'))
#     print(__message('Written to {}'.format(path)))
    
def __success(text):
    return '  (SUCC) {}'.format(text).encode('utf-8')
    
def __failure(text):
    return '!!FAIL!! {}'.format(text).encode('utf-8')
    
def __warning(text):
    return '??WARN?? {}'.format(text).encode('utf-8')
    
def __message(text):
    return '   |MSG| {}'.format(text).encode('utf-8')

## RogueTemple Influences

The data already comes with existing influences documented by the community.

We also load the collected corpus.

In [522]:
roguelikes = pd.read_csv(os.path.join(os.getcwd(), 'roguelikes.csv'), skip_blank_lines=True)
roguelike_names = set(x for x in roguelikes['Name'] if isinstance(x, str))

roguelikelikes = pd.read_csv(os.path.join(os.getcwd(), 'roguelike-likes.csv'), skip_blank_lines=True)
roguelikelike_names = set(x for x in roguelikelikes['Name'] if isinstance(x, str))

roguelike_corpus = read_json('corpus.json')
roguelikelike_corpus = read_json('corpus-roguelike-like.json')

### RogueTemple Influences

In [523]:
import datetime

roguelike_universe_data = roguelikes.copy(deep=True)
roguelike_universe_data.set_index('Name')

for i, roguelike in roguelike_universe_data.iterrows():
    title = roguelike['Name']
    influences = roguelike['Influences']
    
    if not isinstance(title, str):
        continue
        
    if isinstance(influences, str):
        roguelike['Influences'] = [x.strip() for x in influences.split(',')]
    else:
        roguelike['Influences'] = []
        
    if isinstance(roguelike['Released'], str):
        year, month, day = (int(x) for x in roguelike['Released'].split('/'))
        if year == 1000:
            roguelike['Released'] = None
        else:
            try:
                roguelike['Released'] = datetime.date(year, month, day)
            except:
                try:
                    roguelike['Released'] = datetime.date(year, 1, 1)
                except:
                    roguelike['Released'] = None
        
    if isinstance(roguelike['Updated'], str):
        if year == 1000:
            roguelike['Updated'] = None
        else:
            try:
                roguelike['Updated'] = datetime.date(year, month, day)
            except:
                try:
                    roguelike['Updated'] = datetime.date(year, 1, 1)
                except:
                    roguelike['Updated'] = None
        
# sample of influences
roguelike_universe_data.head(10)

,Name,RogueTemple,Link,Status,Released,Updated,Developer,Theme,Influences
0,100 Heroes: Shopkeeper of Doom,http://roguebasin.roguelikedevelopment.org/ind...,http://www.bay12forums.com/smf/index.php?topic...,alpha,2012-01-01,2012-01-01,Paul Wright,Economics/Trading,[Recettear]
1,100 Rogues,http://roguebasin.roguelikedevelopment.org/ind...,http://www.100rogues.com/,stable,2010-05-06,2010-05-06,Dinofarm Games,Fantasy,[Rogue]
2,1Quest,http://roguebasin.roguelikedevelopment.org/ind...,http://www.ratzngodz.fr/,stable,2014-02-20,2014-02-20,Ratz 'N' Godz,Fantasy,"[Dungeon Crawl Stone Soup, Dominion4]"
3,3059,http://roguebasin.roguelikedevelopment.org/ind...,https://sites.google.com/site/free3069/3059---...,stable,2005-01-01,2005-01-01,Phr00t,"Science Fiction, Alien Planets, Futuristic",[NetHack]
4,3069,http://roguebasin.roguelikedevelopment.org/ind...,http://sites.google.com/site/free3069/,stable,2009-07-06,2009-07-06,Phr00t,"Science Fiction, Alien Planets, Futuristic",[3059]
5,3079,http://roguebasin.roguelikedevelopment.org/ind...,http://sites.google.com/site/3079game/,stable,2011-10-25,2011-10-25,Phr00t,"Science Fiction, Alien Planets, Futuristic","[3059, 3069, Fallout, Minecraft]"
6,3089,http://roguebasin.roguelikedevelopment.org/ind...,http://3089game.wordpress.com/,stable,2013-02-02,2013-02-02,Phr00t,"Science Fiction, Alien Planets, Futuristic","[3059, 3069, 3079, Borderlands]"
7,7KBRLL,http://roguebasin.roguelikedevelopment.org/ind...,https://sites.google.com/site/7kbrll,stable,2014-07-11,2014-07-11,OMVTW,"humor, psychological","[NetHack, Rogue]"
8,91,http://roguebasin.roguelikedevelopment.org/ind...,http://startcontinue.com/,beta,None,None,Greg Smith,Modern,[]
9,@Star Wars,http://roguebasin.roguelikedevelopment.org/ind...,http://www.caffeineoverdose.me/,alpha,2012-01-01,2012-01-01,Andrew Wright (aka roocey),"Science Fiction, Space",[Crawl]


### Keyword spotting: matching mentions of other games

In [524]:
not_games = set([
    'Steam',
    'Infinite Space', # Not related to the roguelike game here
    'There',
    'Android',
    'Androids',
    'Combat',
    'How',
    'Core',
    'Add',
    'Monsters',
    'Ghosts',
    'Epic',
    'Adventure', # Usually referred to as Colossal Cave Adventure
    'Contact',
    'Change',
    'Corruption',
    'Ancient Domains of Mystery', # Already covered in other places
    'Dungeons',
    'One',
    'Meanwhile',
    'Barbarian',
    'Shift',
    'Which',
    'Creatures',
    'Dollar',
    'Underground',
    'Curses',
    'Air',
    'Trader',
    'Files',
    'Space',
    'Go',
    'Bonus',
    'Ogre',
    'Legendary',
    'Tower',
    'Wizard',
    'January',
    'Fantasy',
    'Deep',
    'Assault',
    'Alchemist',
    'Speed',
    'File',
    'Elf',
    'Plot',
    'Chaos',
    'Warlord',
    'Where',
    'Chrome',
    'Dexterity',
    'Blood',
    'White',
    'Charly',
    'Scout',
    'Berserk',
    'Clans',
    'Heretic',
    'Dungeon',
    'Satyr',
    'Spider',
    'Lair',
    'Break',
    'AWOL',
    'Create',
    'Campaign',
    'Music',
    'Pop',
    'Falcon',
    'Escape',
    'Elves',
    'Fred',
    'Life',
    'Joshua',
    'Up',
    'ER',
    'Ki',
    'VC',
    'Veteran',
    'People',
    'Columns',
    'Explore',
    'Town',
    'Attack',
    'Firefox',
    'Grow',
    'Search',
    'Satan',
    'Time',
    'Pacific',
    'Black',
    'Love',
    'Race',
    'Silver',
    'Henry',
])

In [525]:
import re
from collections import Counter

roguelike_influence = []
roguelikelike_influence = []
other_influence = []

for i, roguelike in enumerate(roguelike_corpus):
    roguelike_things = Counter()
    roguelikelike_things = Counter()
    other_things = Counter()
    things = []
    
    if isinstance(roguelike['title'], str):
        for webpage in roguelike['text']:
            for paragraph in webpage.split('\n\n'):
                if not paragraph.strip():
                    continue

                current = ''
                for token in re.split('\W', paragraph):
                    if      len(token) > 1 and \
                            re.compile("^[A-Z0-9][\w:']*[\w:']|[A-Z\.]+$").match(token) or \
                            (current and token in ('the', 'of', 'no', 'to')):
                        current += '{} '.format(token)
                    elif current:
                        things.append(current.strip())
                        current = ''

        for x in things:
            if x in roguelike_names and x != roguelike['title'] and len(x) > 2:
                roguelike_things[x] += 1
            elif x in roguelikelike_names and x != roguelike['title'] and len(x) > 2:
                roguelikelike_things[x] += 1
            elif x in video_game_names and x != roguelike['title'] and x not in not_games and len(x) > 2:
                other_things[x] += 1  
            
    roguelike_influence.append({
        "title": roguelike['title'],
        "influences": roguelike_things
    })
    
    roguelikelike_influence.append({
        "title": roguelike['title'],
        "influences": roguelikelike_things
    })
    
    other_influence.append({
        "title": roguelike['title'],
        "influences": other_things
    })
    
ri = [[y[0] for y in x["influences"].most_common(5)] for x in roguelike_influence]
rli = [[y[0] for y in x["influences"].most_common(5)] for x in roguelikelike_influence]
oi = [[y[0] for y in x["influences"].most_common(5)] for x in other_influence]

roguelike_universe_data["Inferred_Roguelike_Influences"] = ri
roguelike_universe_data["Inferred_Roguelikelike_Influences"] = rli
roguelike_universe_data["Inferred_Other_Influences"] = oi

roguelike_universe_data

,Name,RogueTemple,Link,Status,Released,Updated,Developer,Theme,Influences,Inferred_Roguelike_Influences,Inferred_Roguelikelike_Influences,Inferred_Other_Influences
0,100 Heroes: Shopkeeper of Doom,http://roguebasin.roguelikedevelopment.org/ind...,http://www.bay12forums.com/smf/index.php?topic...,alpha,2012-01-01,2012-01-01,Paul Wright,Economics/Trading,[Recettear],[],[],[]
1,100 Rogues,http://roguebasin.roguelikedevelopment.org/ind...,http://www.100rogues.com/,stable,2010-05-06,2010-05-06,Dinofarm Games,Fantasy,[Rogue],"[Rogue, Diablo, Cardinal Quest, The Dungeon, S...",[],"[Persona, Tetris, Galaga, Star Wars, The Masqu..."
2,1Quest,http://roguebasin.roguelikedevelopment.org/ind...,http://www.ratzngodz.fr/,stable,2014-02-20,2014-02-20,Ratz 'N' Godz,Fantasy,"[Dungeon Crawl Stone Soup, Dominion4]",[Clarion],[],"[Post Mortem, Hotel, Suspect, Knock, Quest of ..."
3,3059,http://roguebasin.roguelikedevelopment.org/ind...,https://sites.google.com/site/free3069/3059---...,stable,2005-01-01,2005-01-01,Phr00t,"Science Fiction, Alien Planets, Futuristic",[NetHack],"[3079, 3089, 3069, Sword of Fargoal, Rogue]",[],"[Phantom Crash, HeroQuest, Wizardry, Minecraft..."
4,3069,http://roguebasin.roguelikedevelopment.org/ind...,http://sites.google.com/site/free3069/,stable,2009-07-06,2009-07-06,Phr00t,"Science Fiction, Alien Planets, Futuristic",[3059],"[3089, 3059, 3079, Sword of Fargoal, Decker]",[],"[Image, Descent, HeroQuest, Animal, Battle of ..."
5,3079,http://roguebasin.roguelikedevelopment.org/ind...,http://sites.google.com/site/3079game/,stable,2011-10-25,2011-10-25,Phr00t,"Science Fiction, Alien Planets, Futuristic","[3059, 3069, Fallout, Minecraft]","[3089, 3059, 3069]",[],"[Minecraft, Gentrieve, Fallout, Deus Ex, Borde..."
6,3089,http://roguebasin.roguelikedevelopment.org/ind...,http://3089game.wordpress.com/,stable,2013-02-02,2013-02-02,Phr00t,"Science Fiction, Alien Planets, Futuristic","[3059, 3069, 3079, Borderlands]","[3079, 3059, 3069]",[],[Overlord]
7,7KBRLL,http://roguebasin.roguelikedevelopment.org/ind...,https://sites.google.com/site/7kbrll,stable,2014-07-11,2014-07-11,OMVTW,"humor, psychological","[NetHack, Rogue]",[NetHack],[],[]
8,91,http://roguebasin.roguelikedevelopment.org/ind...,http://startcontinue.com/,beta,None,None,Greg Smith,Modern,[],[],[],[]
9,@Star Wars,http://roguebasin.roguelikedevelopment.org/ind...,http://www.caffeineoverdose.me/,alpha,2012-01-01,2012-01-01,Andrew Wright (aka roocey),"Science Fiction, Space",[Crawl],[],[],"[Star Wars, Crawl, Balance]"


In [526]:
import math

roguelikelike_universe_data = roguelikelikes.copy(deep=True)
roguelikelike_universe_data.set_index('Name')
roguelikelike_universe_data['Released'] = roguelikelike_universe_data['Released'].astype(datetime.date)
roguelikelike_universe_data['Updated'] = roguelikelike_universe_data['Updated'].astype(datetime.date)
roguelikelike_universe_data['Influences'] = roguelikelike_universe_data['Influences'].astype(list)

for i, roguelike in roguelikelike_universe_data.iterrows():
    title = roguelike['Name']
    influences = roguelike['Influences']
    
    if not isinstance(title, str):
        continue
        
    if isinstance(influences, str):
        roguelike['Influences'] = [x.strip() for x in influences.split(',')]
    else:
        roguelike['Influences'] = []
        
    if not math.isnan(roguelike['Released']):
        roguelike['Released'] = datetime.date(int(roguelike['Released']), 1, 1)
        
    if not math.isnan(roguelike['Updated']):
        roguelike['Updated'] = datetime.date(int(roguelike['Updated']), 1, 1)
        
# sample of influences
roguelikelike_universe_data.head(10)

,Name,Released,Updated,Developer,Theme,Influences
0,ToeJam & Earl,1991-01-01,NaN,Johnson Voorsanger Productions,Fantasy,[]
1,Diablo,1996-01-01,NaN,Blizzard North,Fantasy,[]
2,Diablo II,2000-01-01,NaN,Blizzard Entertainment,Fantasy,[]
3,Lost Labyrinth,2001-01-01,2011-01-01,Lost Labyrinth,Fantasy,[]
4,Strange Adventures In Infinite Space,2002-01-01,2004-01-01,"Rich Carlson, Iikka Keränen",Space science fiction,[]
5,Weird Worlds: Return to Infinite Space,2005-01-01,2014-01-01,"Rich Carlson, Iikka Keränen",Space science fiction,[]
6,Spelunky,2009-01-01,2012-01-01,"Derek Yu, Andy Hull",Fantasy,[]
7,The Binding of Isaac,2011-01-01,NaN,"Edmund McMillen, Florian Himsl","Surrealistic, horror",[]
8,FTL: Faster Than Light,2012-01-01,NaN,Subset Games,Space science fiction,[]
9,Diablo III,2012-01-01,NaN,Blizzard Entertainment,Fantasy,[]


In [527]:
import re
from collections import Counter

roguelike_influence = []
roguelikelike_influence = []
other_influence = []

for i, roguelike in enumerate(roguelikelike_corpus):
    roguelike_things = Counter()
    roguelikelike_things = Counter()
    other_things = Counter()
    things = []
    
    if isinstance(roguelike['title'], str):
        for webpage in roguelike['text']:
            for paragraph in webpage.split('\n\n'):
                if not paragraph.strip():
                    continue

                current = ''
                for token in re.split('\W', paragraph):
                    if      len(token) > 1 and \
                            re.compile("^[A-Z0-9][\w:']*[\w:']|[A-Z\.]+$").match(token) or \
                            (current and token in ('the', 'of', 'no', 'to')):
                        current += '{} '.format(token)
                    elif current:
                        things.append(current.strip())
                        current = ''

        for x in things:
            if x in roguelike_names and x != roguelike['title']:
                roguelike_things[x] += 1
            elif x in roguelikelike_names and x != roguelike['title']:
                roguelikelike_things[x] += 1
            elif x in video_game_names and x != roguelike['title'] and x not in not_games:
                other_things[x] += 1  
            
    roguelike_influence.append({
        "title": roguelike['title'],
        "influences": roguelike_things
    })
    
    roguelikelike_influence.append({
        "title": roguelike['title'],
        "influences": roguelikelike_things
    })
    
    other_influence.append({
        "title": roguelike['title'],
        "influences": other_things
    })
    
ri = [[y[0] for y in x["influences"].most_common(5)] for x in roguelike_influence]
rli = [[y[0] for y in x["influences"].most_common(5)] for x in roguelikelike_influence]
oi = [[y[0] for y in x["influences"].most_common(5)] for x in other_influence]

roguelikelike_universe_data["Inferred_Roguelike_Influences"] = ri
roguelikelike_universe_data["Inferred_Roguelikelike_Influences"] = rli
roguelikelike_universe_data["Inferred_Other_Influences"] = oi
roguelikelike_universe_data.head(10)

,Name,Released,Updated,Developer,Theme,Influences,Inferred_Roguelike_Influences,Inferred_Roguelikelike_Influences,Inferred_Other_Influences
0,ToeJam & Earl,1991-01-01,NaN,Johnson Voorsanger Productions,Fantasy,[],"[Rogue, 91]",[],"[Streets of Rage, Genesis, Starflight, Sonic t..."
1,Diablo,1996-01-01,NaN,Blizzard North,Fantasy,[],"[Rogue, Sanctuary, Telengard]","[Diablo III, Diablo II]","[Hellfire, World of Warcraft, StarCraft, Justi..."
2,Diablo II,2000-01-01,NaN,Blizzard Entertainment,Fantasy,[],"[Diablo, Sanctuary, Rogue, Demon, Cataclysm]",[Diablo III],"[StarCraft, Baal, World of Warcraft, Paladin, ..."
3,Lost Labyrinth,2001-01-01,2011-01-01,Lost Labyrinth,Fantasy,[],[Diablo],[Rogue Legacy],"[Chrono Trigger, Switch, Bermuda Triangle, Mid..."
4,Strange Adventures In Infinite Space,2002-01-01,2004-01-01,"Rich Carlson, Iikka Keränen",Space science fiction,[],"[Rogue, NetHack]",[],"[Star Trek, Dragon, Bejeweled, Tetris, Starfli..."
5,Weird Worlds: Return to Infinite Space,2005-01-01,2014-01-01,"Rich Carlson, Iikka Keränen",Space science fiction,[],"[Fallen, 91]",[],"[Doom, Ripcord, Memento Mori, Inferno, Gods]"
6,Spelunky,2009-01-01,2012-01-01,"Derek Yu, Andy Hull",Fantasy,[],"[Diablo, Hack]",[],"[Braid, Aquaria, Super Meat Boy, Eternal Daugh..."
7,The Binding of Isaac,2011-01-01,NaN,"Edmund McMillen, Florian Himsl","Surrealistic, horror",[],[Rogue],[],"[Super Meat Boy, The Legend of Zelda, NBA, Gen..."
8,FTL: Faster Than Light,2012-01-01,NaN,Subset Games,Space science fiction,[],"[Rogue, Spelunky, Diablo]",[The Binding of Isaac],"[FTL, Federation, Star Wars, Star Trek, Battle..."
9,Diablo III,2012-01-01,NaN,Blizzard Entertainment,Fantasy,[],"[Diablo, Sanctuary, Demon, Rogue, 91]",[Diablo II],"[Necromancer, World of Warcraft, Heaven, Basti..."


## Save the year of publication for each game

In [528]:
ignored = set([
    "Tolkien's Middle-Earth",
    "EmoSquid",
    "City of the Condemned",
    "Crosswords",
    "Older Console RPG",
    "Dark Hersey",
    "Dungeon Monkey",
    "games from 80's",
    "Rogue Rage",
    "Cellular automata",
    "Deep Realm Heroes",
    "Slimy Lichmummy", # Beta
    "Zap'M",
    "some dungeon crawlers",
    "MathRL",
    "H.P. Lovecraft",
    "Horror movies",
    "Monty Python",
    "1984",
    "Doomsday 2000",
    "UnNetHack", # No release year found
    "d20",
    "interactive fiction",
    "interactive fiction and roguelikes",
    "A DAY @ THE ZOO",
    "Ascii Wilderness", # Not major or stable
    "Demon Tactic", # Alpha
    "Monster Caves", # Unable to verify
    "Gem miner",
    "DDRogue",
    "Rogue Crystal Quest",
    "noir movies",
    "oldschool adventure games",
    "miscellaneous fantasy",
    "the Greek alphabet",
    "SporkHack",
    "Anne McCaffrey's Pern",
    "Pern",
    "SLASH",
    "Half-way",
    "Cataclysm: Dark Days Ahead", # NEED TO BE ADDED TO ROGUETEMPLE
    "CataclysmDDA",
    "Western RPGs",
    "The Dungeon of Doom", 
    "Dragon Ball",
    "Christianity",
    "Hneftafl",
    "PabloQuest",
    "President",
    "Colorado",
    "Kudos",
    "Up",
])

video_games = pd.read_json(os.path.join(os.getcwd(), 'games.json'))
video_game_names = set(x for x in video_games['title'] if isinstance(x, str) and not all(d.isdigit() for d in x))

def get_year(title):
    if title in roguelike_names:
        return int(roguelikes.loc[roguelikes['Name'] == title].iloc[0]['Released'][:4])
    elif title in roguelikelike_names:
        return int(roguelikelikes.loc[roguelikelikes['Name'] == title].iloc[0]['Released'])
    elif title in video_game_names:
        return int(video_games.loc[video_games['title'] == title].iloc[0]['year'])
    elif title in ignored:
        return None # Out of scope
    raise Exception("{} has no release year on file.".format(title))

game_year = {}

for influences in roguelike_universe_data['Influences']:
    for game in influences:
#         print(game)
        year = get_year(game)
        if year:
            game_year[game] = year
for influences in roguelike_universe_data['Inferred_Roguelike_Influences']:
    for game in influences:
        year = get_year(game)
        if year:
            game_year[game] = year
for influences in roguelike_universe_data['Inferred_Roguelikelike_Influences']:
    for game in influences:
        year = get_year(game)
        if year:
            game_year[game] = year
for influences in roguelike_universe_data['Inferred_Other_Influences']:
    for game in influences:
        year = get_year(game)
        if year:
            game_year[game] = year
for game in roguelike_universe_data['Name']:
    year = get_year(game)
    if year:
        game_year[game] = year
        
for influences in roguelikelike_universe_data['Influences']:
    for game in influences:
        year = get_year(game)
        if year:
            game_year[game] = year
for influences in roguelikelike_universe_data['Inferred_Roguelike_Influences']:
    for game in influences:
        year = get_year(game)
        if year:
            game_year[game] = year
for influences in roguelikelike_universe_data['Inferred_Roguelikelike_Influences']:
    for game in influences:
        year = get_year(game)
        if year:
            game_year[game] = year
for influences in roguelikelike_universe_data['Inferred_Other_Influences']:
    for game in influences:
        year = get_year(game)
        if year:
            game_year[game] = year
for game in roguelikelike_universe_data['Name']:
    year = get_year(game)
    if year:
        game_year[game] = year
        
print(len(game_year))

1525


## Convert Lists to JSON

In [529]:
import json

def list_to_JSON(xs):
    return [json.dumps(x) for x in xs]

roguelike_universe_data["Influences"] = list_to_JSON(roguelike_universe_data["Influences"])
roguelike_universe_data["Inferred_Roguelike_Influences"] = list_to_JSON(roguelike_universe_data["Inferred_Roguelike_Influences"])
roguelike_universe_data["Inferred_Roguelikelike_Influences"] = list_to_JSON(roguelike_universe_data["Inferred_Roguelikelike_Influences"])
roguelike_universe_data["Inferred_Other_Influences"] = list_to_JSON(roguelike_universe_data["Inferred_Other_Influences"])                           
    
roguelikelike_universe_data["Influences"] = list_to_JSON(roguelikelike_universe_data["Influences"])
roguelikelike_universe_data["Inferred_Roguelike_Influences"] = list_to_JSON(roguelikelike_universe_data["Inferred_Roguelike_Influences"])
roguelikelike_universe_data["Inferred_Roguelikelike_Influences"] = list_to_JSON(roguelikelike_universe_data["Inferred_Roguelikelike_Influences"])
roguelikelike_universe_data["Inferred_Other_Influences"] = list_to_JSON(roguelikelike_universe_data["Inferred_Other_Influences"])


## Save contents to file for visualisation


In [530]:
roguelike_universe_data.to_csv('roguelike-influence.csv')
roguelikelike_universe_data.to_csv('roguelikelike-influence.csv')
save_json('games-influence.json', game_year)